# **Space X  Falcon 9 First Stage Landing Prediction**
## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia

In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches

  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame

In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests

In [3]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table

In [4]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on 
`9th June 2021`

In [6]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object

### TASK 1: Request the Falcon9 Launch Wiki page from its URL
First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.

In [8]:
# TASK 1: Request the Falcon9 Launch Wiki page from its URL
# Use requests.get() method with the provided static_url
# Assign the response to an object

# Enviar la solicitud HTTP GET
response = requests.get(static_url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    print("Request was successful!")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

# Almacenar el contenido HTML en una variable
html_content = response.text  # El contenido HTML como texto

Request was successful!


Create a `BeautifulSoup` object from the HTML `response`


In [9]:
# Importar la clase BeautifulSoup del módulo bs4
from bs4 import BeautifulSoup

# Crear un objeto BeautifulSoup a partir del contenido HTML de la respuesta
soup = BeautifulSoup(html_content, 'html.parser')

# Imprimir el título de la página para verificar que el objeto se creó correctamente
print(soup.title.string)


List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header

Next, we want to collect all relevant column names from the HTML table header

Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [12]:
#puedes usar el método find_all de BeautifulSoup para buscar todos los elementos de tipo <table> en la página HTML.
# Buscar todas las tablas en la página HTML
html_tables = soup.find_all('table')

# Imprimir la cantidad de tablas encontradas para verificar
print(f"Número de tablas encontradas: {len(html_tables)}")


Número de tablas encontradas: 26


Starting from the third table is our target table contains the actual launch records.

In [20]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
#print(first_launch_table)

Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [21]:
# Inicializamos la lista para almacenar los nombres de las columnas
column_names = []

# Usamos find_all() para buscar todos los elementos <th> en la tabla seleccionada
table_headers = first_launch_table.find_all('th')

# Iteramos sobre cada elemento <th>
for header in table_headers:
    # Extraemos el nombre de la columna usando extract_column_from_header()
    column_name = extract_column_from_header(header)
    
    # Si el nombre no está vacío y tiene longitud mayor a 0, lo agregamos a la lista
    if column_name is not None and len(column_name) > 0:
        column_names.append(column_name)

In [22]:
# Imprimimos los nombres de las columnas para verificar
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables
We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [79]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.

Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.

To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [94]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            #print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            #customer = row[6].a.string
            #print(customer)
            if row[6].a and row[6].a.string:
                customer = row[6].a.string.strip()  # Extrae el texto del enlace y elimina espacios.
            else:
                customer = None
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)

            # Agregar los valores a launch_dict
            launch_dict['Flight No.'].append(flight_number)
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)
            launch_dict['Version Booster'].append(bv)
            launch_dict['Launch site'].append(launch_site)
            launch_dict['Payload'].append(payload)
            launch_dict['Payload mass'].append(payload_mass)
            launch_dict['Orbit'].append(orbit)
            launch_dict['Customer'].append(customer)
            launch_dict['Launch outcome'].append(launch_outcome)
            launch_dict['Booster landing'].append(booster_landing)

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.

In [95]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [96]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,121,CCSFS,SXM-8,"7,000 kg",GTO,Sirius XM,Success\n,F9 B5,Success,6 June 2021,04:26
1,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.07B0003.18,Failure,4 June 2010,18:45
2,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.07B0004.18,Failure,8 December 2010,15:43
3,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
4,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.07B0006.18,No attempt,8 October 2012,00:35


In [97]:
df.to_csv('spacex_web_scraped.csv', index=False)

<hr>
<hr>

In [99]:
import requests
import pandas as pd

# Perform a GET request on the SpaceX API
url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)

# Convert the response to JSON
data = response.json()

# Normalize the JSON data into a DataFrame
df = pd.json_normalize(data)

# Extract the year from the 'static_fire_date_utc' column (first row)
# Ensure that the static_fire_date_utc is in datetime format first
df['static_fire_date_utc'] = pd.to_datetime(df['static_fire_date_utc'], errors='coerce')

# Extract the year from the first row
first_row_year = df['static_fire_date_utc'].iloc[0].year

# Print the year
print(first_row_year)


2006


In [134]:
# Provided IDs for Falcon 1 and Falcon 9
falcon_1_id = '5e9d0d95eda69955f709d1eb'  # Falcon 1 rocket ID
falcon_9_id = '5e9d0d95eda69973a809d1ec'  # Falcon 9 rocket ID

# Filter Falcon 9 launches
falcon_9_launches = df[df['rocket'] == falcon_9_id]

# Filter out Falcon 1 launches from the DataFrame
df_without_falcon_1 = df[df['rocket'] != falcon_1_id]

# Count the Falcon 9 launches in the filtered DataFrame
falcon_9_after_removal = df_without_falcon_1[df_without_falcon_1['rocket'] == falcon_9_id]

# Get the count of Falcon 9 launches
falcon_9_count = falcon_9_after_removal.shape[0]

print(f"Number of Falcon 9 launches after removing Falcon 1 launches: {falcon_9_count}")

Number of Falcon 9 launches after removing Falcon 1 launches: 195


In [127]:
df['columna_sin_duplicados'] = df['rocket'].drop_duplicates()

In [131]:
df_unique = df.drop_duplicates(subset=['rocket'])

In [132]:
df_unique.head()

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,...,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings,columna_sin_duplicados
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],...,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-fa...,https://en.wikipedia.org/wiki/DemoSat,NaN,5e9d0d95eda69955f709d1eb
5,2010-03-13T00:00:00.000Z,1.268438e+09,False,0.0,5e9d0d95eda69973a809d1ec,True,[],None,[],[],...,None,[],[],http://forum.nasaspaceflight.com/index.php?act...,https://www.youtube.com/watch?v=nxSxgBKlYws,nxSxgBKlYws,http://www.spacex.com/news/2013/02/12/falcon-9...,https://en.wikipedia.org/wiki/Dragon_Spacecraf...,NaN,5e9d0d95eda69973a809d1ec
54,2018-01-24T17:30:00.000Z,1.516815e+09,False,9000.0,5e9d0d95eda69974db09d1ed,True,[],"The launch was a success, and the side booster...",[],"[5ea6ed2f080df4000697c90c, 5ea6ed2f080df400069...",...,None,[],[https://farm5.staticflickr.com/4745/401103041...,http://www.spacex.com/sites/spacex/files/falco...,https://www.youtube.com/watch?v=wbSwFU6tY1c,wbSwFU6tY1c,https://spaceflightnow.com/2018/02/07/spacex-d...,https://en.wikipedia.org/wiki/Elon_Musk%27s_Te...,NaN,5e9d0d95eda69974db09d1ed


In [133]:
# Establece el número máximo de caracteres para mostrar en una columna
pd.set_option('display.max_colwidth', None)

# Ahora puedes ver todas las columnas con texto completo
print(df_unique['links.article'])

0                                              https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html
5                                                                   http://www.spacex.com/news/2013/02/12/falcon-9-flight-1
54    https://spaceflightnow.com/2018/02/07/spacex-debuts-worlds-most-powerful-rocket-sends-tesla-toward-the-asteroid-belt/
Name: links.article, dtype: object
